# Cleaning and F.E.

In [167]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smfm
import patsy
import pprint
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from math import sqrt
from sklearn import ensemble

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, LogisticRegression
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier, BaggingRegressor
from sklearn.neighbors import KNeighborsClassifier
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import SelectFromModel, RFE, SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier

import xgboost as xgb
import uuid 
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', '{:.2f}'.format)
np.set_printoptions(suppress=True)

In [3]:
df = pd.read_csv("./data/traffic_pop_data.csv")

##### Binning

In [4]:
df_prune = df.copy()

In [5]:
def Mon(x):
    return x.split(" ")[0].split('-')[1]
def Day(x):
    return x.split(" ")[0].split('-')[2]
def Hour(x):
    return x.split(" ")[1].split(':')[0]
def Minute(x):
    return x.split(" ")[1].split(':')[1]

In [6]:
df_prune['StartMonth'] = df_prune['StartTime(UTC)'].apply(lambda x: Mon(x)).astype(int)
df_prune['StartDay'] = df_prune['StartTime(UTC)'].apply(lambda x: Day(x)).astype(int)
df_prune['StartHour'] = df_prune['StartTime(UTC)'].apply(lambda x: Hour(x)).astype(int)
df_prune['StartMinute'] = df_prune['StartTime(UTC)'].apply(lambda x: Minute(x)).astype(int)

In [7]:
df_prune['StartTime(UTC)'] = pd.to_datetime(df_prune['StartTime(UTC)'])

def get_dayofweek(x):
    return x.dayofweek
df_prune['DayOfWeek'] = df_prune['StartTime(UTC)'].apply(lambda x: get_dayofweek(x)).astype(int)

In [8]:
df_prune.drop('StartTime(UTC)', inplace=True, axis = 1)

In [9]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_prune['County'] = le.fit_transform(df_prune['County'])

In [10]:
# (month:label) --- (1-3 : 0), (4-6 : 1), (7-9 : 2), (10-12 : 3)
df_prune['Season'] = np.where((df_prune['StartMonth']<=3), 0, 
                                  np.where((df_prune['StartMonth']>3) & (df_prune['StartMonth']<=6), 1,
                                          np.where((df_prune['StartMonth']>6) & (df_prune['StartMonth']<=9), 2, 3))
                                  )

In [11]:
# (day:label) --- (1-7 : 0), (8-15 : 1), (16-23 : 2), (24-28+ : 3)
df_prune['WeekofMonth'] = np.where((df_prune['StartDay']<=7), 0, 
                                  np.where((df_prune['StartDay']>8) & (df_prune['StartDay']<=15), 1,
                                          np.where((df_prune['StartDay']>16) & (df_prune['StartDay']<=23), 2, 3))
                                  )

In [12]:
# (hour:label) --- (1-6 : 0), (7-12 : 1), (13-18 : 2), (19-24 : 3)
df_prune['HourOfDay'] = np.where((df_prune['StartHour']<=7), 0, 
                                  np.where((df_prune['StartHour']>8) & (df_prune['StartHour']<=15), 1,
                                          np.where((df_prune['StartHour']>16) & (df_prune['StartHour']<=23), 2, 3))
                                  )

In [13]:
# (minute:label) --- (1-15 : 0), (16-30 : 1), (31-45 : 2), (46-60 : 3)
df_prune['MinuteOfHour'] = np.where((df_prune['StartMinute']<=7), 0, 
                                  np.where((df_prune['StartMinute']>8) & (df_prune['StartMinute']<=15), 1,
                                          np.where((df_prune['StartMinute']>16) & (df_prune['StartMinute']<=23), 2, 3))
                                  )

In [14]:
df_prune.drop(['irs_estimated_population_2015', 
               'Population Estimate (as of July 1) - 2018 - Male; Median age (years)', 
               'Population Estimate (as of July 1) - 2018 - Female; Median age (years)', 
               'StartMonth', 
               'StartDay', 
               'StartHour', 
               'StartMinute',
              'County'], 
              axis = 1,
              inplace=True)

In [15]:
df_prune.head()

,Severity,LocationLat,LocationLng,ZipCode,Duration,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years),Density(/sqmi),DayOfWeek,Season,WeekofMonth,HourOfDay,MinuteOfHour
0,1,47.01,-122.91,98501,80,39.10,335.30,2,3,3,2,3
1,2,47.61,-122.33,98104,12,36.90,870.90,3,3,0,0,3
2,1,47.96,-122.20,98203,18,38.00,334.80,3,3,0,0,3
3,1,47.56,-122.19,98006,22,36.90,870.90,3,3,0,0,0
4,1,47.62,-122.33,98101,12,36.90,870.90,3,3,0,0,2


##### Encoding

In [100]:
df_enc = df.copy()

In [101]:
def Mon(x):
    return x.split(" ")[0].split('-')[1]
def Day(x):
    return x.split(" ")[0].split('-')[2]
def Hour(x):
    return x.split(" ")[1].split(':')[0]
def Minute(x):
    return x.split(" ")[1].split(':')[1]

df_enc['StartMonth'] = df_enc['StartTime(UTC)'].apply(lambda x: Mon(x)).astype(int)
df_enc['StartDay'] = df_enc['StartTime(UTC)'].apply(lambda x: Day(x)).astype(int)
df_enc['StartHour'] = df_enc['StartTime(UTC)'].apply(lambda x: Hour(x)).astype(int)
df_enc['StartMinute'] = df_enc['StartTime(UTC)'].apply(lambda x: Minute(x)).astype(int)

df_enc['StartTime(UTC)'] = pd.to_datetime(df_enc['StartTime(UTC)'])

def get_dayofweek(x):
    return x.dayofweek
df_enc['DayOfWeek'] = df_enc['StartTime(UTC)'].apply(lambda x: get_dayofweek(x)).astype(int)

df_enc.drop('StartTime(UTC)', inplace=True, axis = 1)

In [102]:
df_enc.head()

,Severity,LocationLat,LocationLng,ZipCode,Duration,irs_estimated_population_2015,County,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years),Population Estimate (as of July 1) - 2018 - Male; Median age (years),Population Estimate (as of July 1) - 2018 - Female; Median age (years),Density(/sqmi),StartMonth,StartDay,StartHour,StartMinute,DayOfWeek
0,1,47.01,-122.91,98501,80,37370,Thurston,39.10,37.70,40.50,335.30,11,30,23,35,2
1,2,47.61,-122.33,98104,12,8990,King,36.90,36.20,37.70,870.90,12,1,0,52,3
2,1,47.96,-122.20,98203,18,32440,Snohomish,38.00,37.20,38.80,334.80,12,1,1,8,3
3,1,47.56,-122.19,98006,22,39630,King,36.90,36.20,37.70,870.90,12,1,1,2,3
4,1,47.62,-122.33,98101,12,10910,King,36.90,36.20,37.70,870.90,12,1,1,19,3


In [103]:
df_enc['isWeekday'] = np.where(df_enc['DayOfWeek'] <=4, 1, 0)

In [104]:
df_enc['Mon'] = np.where(df_enc['DayOfWeek'] ==0, 1, 0)
df_enc['Tue'] = np.where(df_enc['DayOfWeek'] ==1, 1, 0)
df_enc['Wed'] = np.where(df_enc['DayOfWeek'] ==2, 1, 0)
df_enc['Thu'] = np.where(df_enc['DayOfWeek'] ==3, 1, 0)
df_enc['Fri'] = np.where(df_enc['DayOfWeek'] ==4, 1, 0)
df_enc['Sat'] = np.where(df_enc['DayOfWeek'] ==5, 1, 0)
df_enc['Sun'] = np.where(df_enc['DayOfWeek'] ==6, 1, 0)

In [105]:
df_enc['Spring'] = np.where((df_enc['StartMonth'] ==1) | (df_enc['StartMonth'] ==2) | (df_enc['StartMonth'] ==3), 1, 0)
df_enc['Summer'] = np.where((df_enc['StartMonth'] ==4) | (df_enc['StartMonth'] ==5) | (df_enc['StartMonth'] ==6), 1, 0)
df_enc['Autum'] = np.where((df_enc['StartMonth'] ==7) | (df_enc['StartMonth'] ==8) | (df_enc['StartMonth'] ==9), 1, 0)
df_enc['Winter'] = np.where((df_enc['StartMonth'] ==10) | (df_enc['StartMonth'] ==11) | (df_enc['StartMonth'] ==12), 1, 0)

In [106]:
df_enc['Morning(6-12)'] = np.where((df_enc['StartHour'] >= 6) & (df_enc['StartHour'] < 12), 1, 0)
df_enc['Noon(12-18)'] = np.where((df_enc['StartHour'] >= 12) & (df_enc['StartHour'] < 18), 1, 0)
df_enc['Night(18-24)'] = np.where((df_enc['StartHour'] >= 18) & (df_enc['StartHour'] < 24), 1, 0)
df_enc['midNight(24-6)'] = np.where((df_enc['StartHour'] >= 0) & (df_enc['StartHour'] < 6), 1, 0)

In [107]:
df_enc['firstHalfHour'] = np.where((df_enc['StartMinute'] <=30), 1, 0)
df_enc['SecondHalfHour'] = np.where((df_enc['StartMinute'] > 30), 1, 0)

In [108]:
df_enc.drop(['irs_estimated_population_2015', 
               'Population Estimate (as of July 1) - 2018 - Male; Median age (years)', 
               'Population Estimate (as of July 1) - 2018 - Female; Median age (years)', 
               'StartMonth', 
               'StartDay', 
               'StartHour', 
               'StartMinute',
              'County',
            'DayOfWeek'], 
              axis = 1,
              inplace=True)

## setting up pipline

In [16]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [37]:
def boosting_pipeline(X_train, y_train, X_validate, y_validate):
    
    est = HistGradientBoostingRegressor(max_iter =100, 
                                    min_samples_leaf = 24, 
                                    max_leaf_nodes =300, 
                                    l2_regularization = 0.5, 
                                    verbose=0,).fit(X_train, y_train)
    print("Score: ",est.score(X_validate, y_validate))
    
#     result = permutation_importance(est, X_validate, y_validate, n_repeats=10,
#                                 random_state=42, n_jobs=1)
    
    mse = mean_squared_error(y_validate, est.predict(X_validate))
    print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
    
    rms = sqrt(mse)
    print('RMSE:', rms, '\n')
    
#     sorted_idx = result.importances_mean.argsort()

#     fig, ax = plt.subplots(figsize=(15, 10))
#     ax.boxplot(result.importances[sorted_idx].T,
#                vert=False, labels=X_test.columns[sorted_idx])
#     ax.set_title("Permutation Importances (validation set)")
#     fig.tight_layout()
#     plt.show()
    

    
    return est

### F.E. piplines

In [18]:
def explore_fe(df, target):
    ''' 
    A function to do exploratory feature engineering. 
    It's flexible in its purpose, and is currently configured for this project only.

    Inputs:
    df (like X) = Your dataset without the target (y)
    target (like y) = Your target, whatever you are trying to predict ---Binary.

    Output:
    Returns engineered X (dataframe without target) based on the engineering logic.
    '''
    df = df.astype(float)
    df = df.replace({0:1 , 1:2})
    
    for i in range (0, len(df.columns)):
        df[f'{df.columns[i]}^2'] = np.square(df[df.columns[i]])
        df[f'{df.columns[i]}^1/2'] = np.sqrt(df[df.columns[i]])
        df[f'{df.columns[i]} * {df.columns[i+1]}'] = df[df.columns[i]] * df[df.columns[i+1]]
        df[f'{df.columns[i]} / {df.columns[i+1]}'] = df[df.columns[i]] / df[df.columns[i+1]]
#         df[f'{df.columns[i]} + {df.columns[i+1]}'] = df[df.columns[i]] + df[df.columns[i+1]]
#         df[f'{df.columns[i]} - {df.columns[i+1]}'] = df[df.columns[i]] - df[df.columns[i+1]]
        

#     df.fillna(0, inplace = True)
#     df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    df[~df.isin([np.nan, np.inf, -np.inf]).any(1)].astype(np.float64)
    
    X,y= df, target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=4444)
    
    ran = RandomForestClassifier(random_state=5)
    ran.fit(X_train, y_train)
    
    print ('Accuracy: ', accuracy_score(y_test, ran.predict(X_test)))
    print("Precision: {:6.4f},   Recall: {:6.4f},   f1: {:6.4f}".format(precision_score(y_test, ran.predict(X_test)), 
                                                 recall_score(y_test, ran.predict(X_test)), f1_score(y_test, ran.predict(X_test))), '\n')
    
    k = list(X.columns)
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(sorted(list(zip(k, ran.feature_importances_)), key=lambda x: x[1], reverse=True))
    
    return df

In [19]:
def PolynomialFeatures_labeled(input_df,power):
    '''
    Basically this is a cover for the sklearn preprocessing function. 
    The problem with that function is if you give it a labeled dataframe, it ouputs an unlabeled dataframe with potentially
    a whole bunch of unlabeled columns. 

    Inputs:
    input_df = Your labeled pandas dataframe (list of x's not raised to any power) 
    power = what order polynomial you want variables up to. (use the same power as you want entered into pp.PolynomialFeatures(power) directly)

    Output:
    Output: This function relies on the powers_ matrix which is one of the preprocessing function's outputs to create logical labels and 
    outputs a labeled pandas dataframe   
    '''
    poly = PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + " x " + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df

### Feature Selection Pipeline

In [ ]:
def feature_selection(X, y, score_to_keep = 5):
    '''
    A function to select features by votes of 6 models who can calculate feature importances.
    Also prints out how many original features there are, how many selected, and a list of selected features.
    Original idea from https://www.kaggle.com/mlwhiz/feature-selection-using-football-data

    Inputs:
    X = Your dataset without the target (y)
    y = Your target, whatever you are trying to predict --- Binary.
    score_to_keep = Pick features that have a 'score_to_keep' amount of votes --- max is 6 votes, default is 5. 
    
    Output:
    Returns selected_X as a dataframe without target(y). 
    '''
    feature_name = list(X.columns)
    num_feats=len(X.columns)
    def cor_selector(X, y,num_feats):
        cor_list = []
        feature_name = X.columns.tolist()
        # calculate the correlation with y for each feature
        for i in X.columns.tolist():
            cor = np.corrcoef(X[i], y)[0, 1]
            cor_list.append(cor)
        # replace NaN with 0
        cor_list = [0 if np.isnan(i) else i for i in cor_list]
        # feature name
        cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
        # feature selection? 0 for not select, 1 for select
        cor_support = [True if i in cor_feature else False for i in feature_name]
        return cor_support, cor_feature
    cor_support, cor_feature = cor_selector(X, y,num_feats)
    
    X_norm = MinMaxScaler().fit_transform(X)
    chi_selector = SelectKBest(chi2, k=num_feats)
    chi_selector.fit(X_norm, y)
    chi_support = chi_selector.get_support()
    chi_feature = X.loc[:,chi_support].columns.tolist()
    
    rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
    rfe_selector.fit(X_norm, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:,rfe_support].columns.tolist()
    
    embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
    embeded_lr_selector.fit(X_norm, y)
    embeded_lr_support = embeded_lr_selector.get_support()
    embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
    
#     embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
#     embeded_rf_selector.fit(X_norm, y)
#     embeded_rf_support = embeded_rf_selector.get_support()
#     embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
    
#     lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
#             reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
#     embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
#     embeded_lgb_selector.fit(X_norm, y)
#     embeded_lgb_support = embeded_lgb_selector.get_support()
#     embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
    
    feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
#                                         'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
    
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    
    
    selected_X = X.copy()
    to_drop = []
    for i in range (0, len(feature_selection_df)):
        if feature_selection_df.Total.values[i] < score_to_keep:
            to_drop.append(feature_selection_df.Feature.values[i])

    selected_X = selected_X.drop(to_drop, axis = 1)
    
    print ("Number of orginal features: ", num_feats)
    print ("Number of selected features: ", len(selected_X.columns), '\n')
    pp = pprint.PrettyPrinter(indent=4)
    print("Selected Features:")
    pp.pprint(list(selected_X.columns))
    
    
    return selected_X

## split df

##### Binned DF (df_prune)

In [75]:
X, y = df_prune.drop('Duration',axis=1), df_prune['Duration']
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.1, random_state=13)

In [76]:
train, validate, test = train_validate_test_split(df_prune)
X_train, y_train = train.drop('Duration',axis=1), train['Duration']
X_validate, y_validate = validate.drop('Duration',axis=1), validate['Duration']
X_test, y_test = test.drop('Duration',axis=1), test['Duration']

In [77]:
X_validate

,Severity,LocationLat,LocationLng,ZipCode,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years),Density(/sqmi),DayOfWeek,Season,WeekofMonth,HourOfDay,MinuteOfHour
284802,1,46.64,-118.87,99343,30.30,66.80,1,0,3,0,3
292194,1,47.35,-120.10,98850,37.60,21.20,3,0,2,1,3
204862,3,47.19,-122.23,98390,36.30,449.90,3,1,1,0,3
274321,1,47.47,-121.77,98045,36.90,870.90,0,0,1,1,3
524513,3,47.67,-122.38,98107,36.90,870.90,2,3,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...
49375,2,47.16,-122.29,98373,36.30,449.90,5,2,1,2,1
502501,2,47.69,-122.18,98033,36.90,870.90,3,3,1,0,3
26232,3,47.57,-122.12,98008,36.90,870.90,3,3,0,2,3
297912,2,47.24,-122.42,98404,36.30,449.90,5,3,0,2,1


##### Encoded DF (df_enc)

In [109]:
X2, y2= df_enc.drop('Duration',axis=1), df_enc['Duration']

In [110]:
train2, validate2, test2 = train_validate_test_split(df_enc)
X_train2, y_train2 = train2.drop('Duration',axis=1), train2['Duration']
X_validate2, y_validate2 = validate2.drop('Duration',axis=1), validate2['Duration']
X_test2, y_test2 = test2.drop('Duration',axis=1), test2['Duration']

In [111]:
X_train2

,Severity,LocationLat,LocationLng,ZipCode,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years),Density(/sqmi),isWeekday,Mon,Tue,Wed,...,Spring,Summer,Autum,Winter,Morning(6-12),Noon(12-18),Night(18-24),midNight(24-6),firstHalfHour,SecondHalfHour
177039,2,47.89,-122.26,98204,38.00,334.80,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1
383550,1,47.44,-122.27,98188,36.90,870.90,1,0,1,0,...,0,0,1,0,0,1,0,0,1,0
376707,2,47.95,-122.10,98290,38.00,334.80,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
543962,1,47.59,-122.18,98005,36.90,870.90,1,0,0,0,...,0,0,1,0,0,1,0,0,0,1
129708,2,47.44,-122.25,98032,36.90,870.90,1,0,0,0,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260157,2,47.68,-122.10,98052,36.90,870.90,1,0,0,1,...,1,0,0,0,0,0,1,0,1,0
154723,1,47.53,-122.20,98056,36.90,870.90,1,1,0,0,...,0,0,1,0,0,0,1,0,0,1
100987,1,47.65,-122.32,98105,36.90,870.90,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
25060,1,47.65,-117.34,99212,37.60,267.80,1,1,0,0,...,0,0,0,1,0,1,0,0,0,1


In [208]:
df_enc_all =df_enc.copy()

In [209]:
df_enc_all['StartTime(UTC)'] = df['StartTime(UTC)']

In [211]:
df_enc_all.to_csv('./data/classi_/df_enc_all.csv', index = False)

## Start Feature Engineering and testing

### Baseline

##### Gradient Boosting - binned

In [24]:
est = boosting_pipeline(X_train, y_train, X_validate, y_validate)

Score:  0.19109010053266895
The mean squared error (MSE) on test set: 16432.3781
RMSE: 128.18883772529657 



In [25]:
perm = PermutationImportance(est, random_state=1).fit(X_validate, y_validate)
eli5.show_weights(perm, feature_names = X_validate.columns.tolist())

Weight,Feature
0.1847 ± 0.0537,DayOfWeek
0.1458 ± 0.0114,LocationLng
0.1082 ± 0.0755,Season
0.1038 ± 0.0226,HourOfDay
0.0958 ± 0.0236,WeekofMonth
0.0840 ± 0.0463,Severity
0.0830 ± 0.0595,LocationLat
0.0650 ± 0.0375,MinuteOfHour
0.0398 ± 0.0182,ZipCode
0.0180 ± 0.0287,Density(/sqmi)


##### Gradient Boosting - encoded

In [112]:
est = boosting_pipeline(X_train2, y_train2, X_validate2, y_validate2)

Score:  0.3799130374772476
The mean squared error (MSE) on test set: 5546.3421
RMSE: 74.47376795384271 



In [113]:
perm = PermutationImportance(est, random_state=1).fit(X_validate2, y_validate2)
eli5.show_weights(perm, feature_names = X_validate2.columns.tolist())

Weight,Feature
0.3884 ± 0.0481,LocationLng
0.3330 ± 0.2720,Mon
0.3296 ± 0.1779,LocationLat
0.2720 ± 0.0448,Morning(6-12)
0.2615 ± 0.2348,Autum
0.2158 ± 0.2249,midNight(24-6)
0.1922 ± 0.2310,Severity
0.1761 ± 0.1563,firstHalfHour
0.1103 ± 0.0229,ZipCode
0.0481 ± 0.0591,Spring


##### XGboost - binned df vs encoded df

In [114]:
boost = xgb.XGBRegressor()
boost.fit(X_train,y_train)

boost2 = xgb.XGBRegressor()
boost2.fit(X_train2,y_train2)

[17:29:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:29:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [115]:
predictions = boost.predict(X_validate)
print( "Score of Binned df: ", r2_score(y_validate,predictions))

predictions = boost2.predict(X_validate2)
print("Score of Encoded df: ", r2_score(y_validate2,predictions))

Score of Binned df:  -0.10659204606946893
Score of Encoded df:  0.13434382154196634


### R1

##### 2nd degree poly - binned df

In [38]:
X_poly_tr = PolynomialFeatures_labeled(X_train, 2)
X_poly_va = PolynomialFeatures_labeled(X_validate, 2)

In [39]:
X_poly_va

,Constant Term,Severity^1,LocationLat^1,LocationLng^1,ZipCode^1,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years)^1,Density(/sqmi)^1,DayOfWeek^1,Season^1,WeekofMonth^1,...,Season^2,Season^1 x WeekofMonth^1,Season^1 x HourOfDay^1,Season^1 x MinuteOfHour^1,WeekofMonth^2,WeekofMonth^1 x HourOfDay^1,WeekofMonth^1 x MinuteOfHour^1,HourOfDay^2,HourOfDay^1 x MinuteOfHour^1,MinuteOfHour^2
0,1.00,3.00,47.88,-122.28,98087.00,38.00,334.80,5.00,1.00,3.00,...,1.00,3.00,0.00,2.00,9.00,0.00,6.00,0.00,0.00,4.00
1,1.00,2.00,47.24,-122.42,98404.00,36.30,449.90,4.00,1.00,2.00,...,1.00,2.00,1.00,1.00,4.00,2.00,2.00,1.00,1.00,1.00
2,1.00,2.00,47.64,-122.32,98102.00,36.90,870.90,4.00,3.00,1.00,...,9.00,3.00,6.00,9.00,1.00,2.00,3.00,4.00,6.00,9.00
3,1.00,3.00,46.56,-122.88,98532.00,42.90,30.90,3.00,0.00,2.00,...,0.00,0.00,0.00,0.00,4.00,2.00,6.00,1.00,3.00,9.00
4,1.00,2.00,47.54,-122.20,98056.00,36.90,870.90,3.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,6.00,9.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111494,1.00,1.00,47.81,-122.10,98296.00,38.00,334.80,0.00,1.00,3.00,...,1.00,3.00,1.00,3.00,9.00,3.00,9.00,1.00,3.00,9.00
111495,1.00,2.00,47.48,-122.20,98057.00,36.90,870.90,0.00,3.00,0.00,...,9.00,0.00,3.00,9.00,0.00,0.00,0.00,1.00,3.00,9.00
111496,1.00,2.00,47.72,-122.34,98133.00,36.90,870.90,5.00,1.00,3.00,...,1.00,3.00,0.00,2.00,9.00,0.00,6.00,0.00,0.00,4.00
111497,1.00,3.00,47.60,-122.34,98104.00,36.90,870.90,1.00,3.00,2.00,...,9.00,6.00,6.00,9.00,4.00,4.00,6.00,4.00,6.00,9.00


In [40]:
est = boosting_pipeline(X_poly_tr, y_train, X_poly_va, y_validate)

Score:  0.20476869350233817
The mean squared error (MSE) on test set: 16154.5081
RMSE: 127.10038589665245 



In [41]:
perm = PermutationImportance(est, random_state=1).fit(X_poly_va, y_validate)
eli5.show_weights(perm, feature_names = X_poly_va.columns.tolist())

Weight,Feature
0.4933 ± 0.0598,LocationLng^1 x HourOfDay^1
0.3354 ± 0.0499,Density(/sqmi)^1 x HourOfDay^1
0.2995 ± 0.0084,LocationLng^1 x Density(/sqmi)^1
0.2659 ± 0.0050,LocationLng^1 x Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years)^1
0.2145 ± 0.0314,Season^1 x HourOfDay^1
0.0809 ± 0.0227,LocationLat^1 x Season^1
0.0753 ± 0.0043,LocationLat^1 x HourOfDay^1
0.0723 ± 0.0018,Severity^1 x LocationLng^1
0.0697 ± 0.0248,Severity^1 x Density(/sqmi)^1
0.0659 ± 0.0206,LocationLng^1 x Season^1


##### 2nd degree poly - Encoded df

In [116]:
X_poly_tr2 = PolynomialFeatures_labeled(X_train2, 2)
X_poly_va2 = PolynomialFeatures_labeled(X_validate2, 2)

In [117]:
X_poly_va2

,Constant Term,Severity^1,LocationLat^1,LocationLng^1,ZipCode^1,Population Estimate (as of July 1) - 2018 - Both Sexes; Median age (years)^1,Density(/sqmi)^1,isWeekday^1,Mon^1,Tue^1,...,Night(18-24)^2,Night(18-24)^1 x midNight(24-6)^1,Night(18-24)^1 x firstHalfHour^1,Night(18-24)^1 x SecondHalfHour^1,midNight(24-6)^2,midNight(24-6)^1 x firstHalfHour^1,midNight(24-6)^1 x SecondHalfHour^1,firstHalfHour^2,firstHalfHour^1 x SecondHalfHour^1,SecondHalfHour^2
0,1.00,3.00,47.95,-122.30,98203.00,38.00,334.80,1.00,1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
1,1.00,3.00,47.69,-122.31,98115.00,36.90,870.90,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
2,1.00,1.00,47.68,-122.54,98110.00,38.90,448.10,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,1.00,3.00,47.82,-122.32,98036.00,38.00,334.80,1.00,0.00,1.00,...,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
4,1.00,3.00,47.38,-122.25,98032.00,36.90,870.90,1.00,0.00,0.00,...,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111494,1.00,1.00,47.34,-122.25,98001.00,36.90,870.90,1.00,0.00,0.00,...,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00
111495,1.00,2.00,47.23,-119.28,98837.00,33.00,32.80,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
111496,1.00,2.00,47.82,-122.32,98036.00,38.00,334.80,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,0.00,0.00
111497,1.00,2.00,48.00,-122.11,98258.00,38.00,334.80,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,0.00,0.00


In [118]:
est2 = boosting_pipeline(X_poly_tr2, y_train2, X_poly_va2, y_validate2)
perm = PermutationImportance(est2, random_state=1).fit(X_poly_va2, y_validate2)
eli5.show_weights(perm, feature_names = X_poly_va2.columns.tolist())

Score:  0.47547439408075054
The mean squared error (MSE) on test set: 4691.5975
RMSE: 68.49523723055934 



Weight,Feature
1.0525 ± 0.1642,Severity^1 x Density(/sqmi)^1
0.9178 ± 0.1103,Severity^1 x LocationLng^1
0.7620 ± 0.0593,LocationLng^1 x Noon(12-18)^1
0.4724 ± 0.0493,LocationLng^1 x ZipCode^1
0.3514 ± 0.0273,Severity^1 x LocationLat^1
0.2833 ± 0.0481,Severity^1 x isWeekday^1
0.2210 ± 0.0192,Mon^1 x Morning(6-12)^1
0.1950 ± 0.0499,Mon^1 x Autum^1
0.1832 ± 0.0090,LocationLat^1 x LocationLng^1
0.1657 ± 0.0128,Density(/sqmi)^1 x Noon(12-18)^1


##### Running poly transformed features in XGboost

In [120]:
boost3 = xgb.XGBRegressor()
boost3.fit(X_poly_tr,y_train)

boost4 = xgb.XGBRegressor()
boost4.fit(X_poly_tr2,y_train2)

[18:02:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:03:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [122]:
predictions = boost3.predict(X_poly_va)
print( "Score of Binned df: ", r2_score(y_validate,predictions))

predictions2 = boost4.predict(X_poly_va2)
print("Score of Encoded df: ", r2_score(y_validate2,predictions2))

Score of Binned df:  -0.08494188758400534
Score of Encoded df:  0.4154649396838269


##### feature selection 1

In [132]:
perm = PermutationImportance(est2, random_state=1).fit(X_poly_va2, y_validate2)
# eli5.show_weights(perm, feature_names = X_poly_va2.columns.tolist())

In [133]:
sel = SelectFromModel(perm, threshold=0.05, prefit=True)
X_trans = sel.transform(X_poly_va2)

In [140]:
sel.get_support()

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False,  True, False,  True, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [138]:
all_features = X_poly_va2.columns.tolist()

In [139]:
selected_features = []
for i in range(len(sel.get_support())):
    if sel.get_support()[i] == True:
        selected_features.append(all_features[i])

In [141]:
selected_features

['LocationLat^1',
 'Severity^1 x LocationLat^1',
 'Severity^1 x LocationLng^1',
 'Severity^1 x Density(/sqmi)^1',
 'Severity^1 x isWeekday^1',
 'LocationLat^1 x LocationLng^1',
 'LocationLat^1 x ZipCode^1',
 'LocationLat^1 x Density(/sqmi)^1',
 'LocationLat^1 x isWeekday^1',
 'LocationLat^1 x SecondHalfHour^1',
 'LocationLng^1 x ZipCode^1',
 'LocationLng^1 x Density(/sqmi)^1',
 'LocationLng^1 x Mon^1',
 'LocationLng^1 x Noon(12-18)^1',
 'ZipCode^1 x Mon^1',
 'Density(/sqmi)^1 x Mon^1',
 'Density(/sqmi)^1 x Noon(12-18)^1',
 'Mon^1 x Autum^1',
 'Mon^1 x Morning(6-12)^1',
 'Spring^1 x Morning(6-12)^1']

In [184]:
X_trans_tr = pd.DataFrame(X_trans, columns = selected_features)

In [183]:
X_trans_tr

,LocationLat^1,Severity^1 x LocationLat^1,Severity^1 x LocationLng^1,Severity^1 x Density(/sqmi)^1,Severity^1 x isWeekday^1,LocationLat^1 x LocationLng^1,LocationLat^1 x ZipCode^1,LocationLat^1 x Density(/sqmi)^1,LocationLat^1 x isWeekday^1,LocationLat^1 x SecondHalfHour^1,LocationLng^1 x ZipCode^1,LocationLng^1 x Density(/sqmi)^1,LocationLng^1 x Mon^1,LocationLng^1 x Noon(12-18)^1,ZipCode^1 x Mon^1,Density(/sqmi)^1 x Mon^1,Density(/sqmi)^1 x Noon(12-18)^1,Mon^1 x Autum^1,Mon^1 x Morning(6-12)^1,Spring^1 x Morning(6-12)^1
0,47.95,143.85,-366.91,1004.40,3.00,-5864.39,4708750.77,16053.38,47.95,0.00,-12010652.81,-40947.49,-122.30,-122.30,98203.00,334.80,334.80,1.00,0.00,0.00
1,47.69,143.06,-366.94,2612.70,3.00,-5832.73,4678800.68,41530.53,47.69,47.69,-12000744.80,-106522.43,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,47.68,47.68,-122.54,448.10,1.00,-5842.78,4677931.99,21365.62,47.68,47.68,-12022436.19,-54910.34,-0.00,-122.54,0.00,0.00,448.10,0.00,0.00,0.00
3,47.82,143.46,-366.95,1004.40,3.00,-5849.06,4687962.21,16009.73,47.82,47.82,-11991486.76,-40951.79,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,47.38,142.14,-366.74,2612.70,3.00,-5792.23,4644877.43,41264.32,47.38,47.38,-11984156.23,-106465.25,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111494,47.34,47.34,-122.25,870.90,1.00,-5787.37,4639602.54,41230.50,47.34,47.34,-11980132.54,-106463.17,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
111495,47.23,94.46,-238.55,65.60,0.00,-5633.52,4668111.84,1549.16,0.00,47.23,-11789023.15,-3912.30,-0.00,-119.28,0.00,0.00,32.80,0.00,0.00,0.00
111496,47.82,95.63,-244.64,669.60,2.00,-5848.82,4687670.85,16008.73,47.82,0.00,-11991738.03,-40952.65,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
111497,48.00,96.01,-244.21,669.60,2.00,-5861.47,4716678.58,16071.40,48.00,0.00,-11997917.09,-40881.18,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [185]:
est2 = boosting_pipeline(X_sel_tr, y_train2, X_sel_val, y_validate2)
perm = PermutationImportance(est2, random_state=1).fit(X_sel_val, y_validate2)
eli5.show_weights(perm, feature_names = X_sel_val.columns.tolist())

Score:  0.4941735641879764
The mean squared error (MSE) on test set: 4524.3436
RMSE: 67.26324086708448 



Weight,Feature
1.8515 ± 0.2483,Severity^1 x Density(/sqmi)^1
1.6844 ± 0.1573,Severity^1 x LocationLng^1
0.4486 ± 0.0704,Mon^1 x Autum^1
0.4444 ± 0.0385,LocationLat^1 x Density(/sqmi)^1
0.4117 ± 0.1136,LocationLat^1 x isWeekday^1
0.3380 ± 0.1386,LocationLat^1 x SecondHalfHour^1
0.2665 ± 0.1011,LocationLng^1 x Density(/sqmi)^1
0.2341 ± 0.0412,Spring^1 x Morning(6-12)^1
0.2290 ± 0.0405,LocationLat^1 x LocationLng^1
0.1686 ± 0.0578,Mon^1 x Morning(6-12)^1


In [146]:
X_sel_tr = X_poly_tr2[selected_features]
X_sel_val = pd.DataFrame(X_trans, columns = selected_features)
X_sel_val

,LocationLat^1,Severity^1 x LocationLat^1,Severity^1 x LocationLng^1,Severity^1 x Density(/sqmi)^1,Severity^1 x isWeekday^1,LocationLat^1 x LocationLng^1,LocationLat^1 x ZipCode^1,LocationLat^1 x Density(/sqmi)^1,LocationLat^1 x isWeekday^1,LocationLat^1 x SecondHalfHour^1,LocationLng^1 x ZipCode^1,LocationLng^1 x Density(/sqmi)^1,LocationLng^1 x Mon^1,LocationLng^1 x Noon(12-18)^1,ZipCode^1 x Mon^1,Density(/sqmi)^1 x Mon^1,Density(/sqmi)^1 x Noon(12-18)^1,Mon^1 x Autum^1,Mon^1 x Morning(6-12)^1,Spring^1 x Morning(6-12)^1
0,47.95,143.85,-366.91,1004.40,3.00,-5864.39,4708750.77,16053.38,47.95,0.00,-12010652.81,-40947.49,-122.30,-122.30,98203.00,334.80,334.80,1.00,0.00,0.00
1,47.69,143.06,-366.94,2612.70,3.00,-5832.73,4678800.68,41530.53,47.69,47.69,-12000744.80,-106522.43,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,47.68,47.68,-122.54,448.10,1.00,-5842.78,4677931.99,21365.62,47.68,47.68,-12022436.19,-54910.34,-0.00,-122.54,0.00,0.00,448.10,0.00,0.00,0.00
3,47.82,143.46,-366.95,1004.40,3.00,-5849.06,4687962.21,16009.73,47.82,47.82,-11991486.76,-40951.79,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,47.38,142.14,-366.74,2612.70,3.00,-5792.23,4644877.43,41264.32,47.38,47.38,-11984156.23,-106465.25,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111494,47.34,47.34,-122.25,870.90,1.00,-5787.37,4639602.54,41230.50,47.34,47.34,-11980132.54,-106463.17,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
111495,47.23,94.46,-238.55,65.60,0.00,-5633.52,4668111.84,1549.16,0.00,47.23,-11789023.15,-3912.30,-0.00,-119.28,0.00,0.00,32.80,0.00,0.00,0.00
111496,47.82,95.63,-244.64,669.60,2.00,-5848.82,4687670.85,16008.73,47.82,0.00,-11991738.03,-40952.65,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00
111497,48.00,96.01,-244.21,669.60,2.00,-5861.47,4716678.58,16071.40,48.00,0.00,-11997917.09,-40881.18,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [194]:
classi_tr = pd.concat([X_sel_tr, y_train2.reset_index(drop=True)], axis = 1)
classi_va = pd.concat([X_sel_val, y_validate2.reset_index(drop=True)], axis = 1)

In [196]:
classi_tr.to_csv('./data/Engineered/classi_tr.csv', index=False)
classi_va.to_csv('./data/Engineered/classi_va.csv', index=False)

In [144]:
y_validate

284802    40
292194    42
204862    43
274321    41
524513    70
          ..
49375     39
502501    95
26232     24
297912    41
308013    42
Name: Duration, Length: 111499, dtype: int64

##### Run selected features through the model again

In [150]:
est_sel = boosting_pipeline(X_sel_tr, y_train2, X_sel_val, y_validate2)

Score:  0.49349379075528277
The mean squared error (MSE) on test set: 4530.4238
RMSE: 67.3084227806085 



In [148]:
boost_sel = xgb.XGBRegressor()
boost_sel.fit(X_sel_tr,y_train2)

[19:17:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [149]:
predictions = boost_sel.predict(X_sel_val)
print( "Score of Binned df: ", r2_score(y_validate2,predictions))

Score of Binned df:  0.24167109827603106


##### Grid Search

https://www.kaggle.com/omarito/gridsearchcv-xgbregressor-0-556-lb

In [151]:
# A parameter grid for XGBoost
# params = {'n_estimators':[100, 200, 300]
#             'min_child_weight':[1,3,6], 
#           'gamma':[i/10.0 for i in range(3,6)],  
#           'subsample':[i/10.0 for i in range(6,11)],
#             'colsample_bytree':[i/10.0 for i in range(6,11)], 
#           'max_depth': [2,10,20],
#          'learning_rate' : [0.01, 0.1, 0.5],
#          'booster' : ['gbtree', 'gblinear '],
#           'reg_alpha' : [0.1, 0.5, 0.9], 
#           'reg_lambda ' : [0.1, 0.5, 0.9],
#           'verbosity' : [1]
#          }

In [162]:
# A parameter grid for XGBoost
params = {'n_estimators':[100, 200, 300],
            'min_child_weight':[11, 13, 15, 17, 19],
          'gamma':[0.15, 0.2, 0.3, 0.5],  
          'subsample':[0.7, 0.8, 0.9, 1],
            'colsample_bytree':[0.4], 
          'max_depth': [20, 21, 22, 23, 24, 25, 26],
         'learning_rate' : [0.08, 0.1, 0.12],
         'booster' : ['gbtree', 'gblinear'],
          'reg_alpha' : [0, 0.01, 0.05], 
          'reg_lambda ' : [0.05, 0.1, 0.15],
          'verbosity' : [1]
         }

In [152]:
xgb_grid = xgb.XGBRegressor(n_jobs=1) 

# grid = GridSearchCV(xgb_grid, params)
# grid.fit(X_sel_tr, y_train2)

# # Print the r2 score
# print("Best Estimator: ", grid.best_estimator_)
# print("r2 score: ", r2_score(y_validate2, grid.best_estimator_.predict(X_sel_val))) 

[19:47:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:48:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:59:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:00:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:00:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:00:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:00:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:00:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:00:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:01:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:01:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[20:11:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:11:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:13:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[20:23:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:23:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:23:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:23:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[20:34:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:35:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:35:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:35:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:35:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:35:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:35:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:36:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:36:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

...

[07:58:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:59:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:00:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:01:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:01:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:03:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:04:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:04:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[08:50:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:51:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:51:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:52:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:53:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:53:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:54:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:55:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:56:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[09:42:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:45:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:45:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:46:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:47:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:47:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[10:35:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:36:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:37:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:37:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:38:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:39:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:40:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:40:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:41:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

KeyboardInterrupt: 

In [163]:
randgrid = RandomizedSearchCV(estimator = xgb_grid, 
                              param_distributions = params, 
                              n_iter = 100, 
                              n_jobs = 3, 
                              random_state =42,
                              cv =5,
                              verbose = 1)

randgrid.fit(X_sel_tr, y_train2)

print("Best Estimator: ", randgrid.best_estimator_)
print("r2 score: ", r2_score(y_validate2, randgrid.best_estimator_.predict(X_sel_val))) 

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 20.5min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 72.0min
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed: 176.7min
[Parallel(n_jobs=3)]: Done 500 out of 500 | elapsed: 206.1min finished


[23:48:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best Estimator:  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0.15,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=21, min_child_weight=15, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.01, reg_lambda=1, reg_lambda =0.05, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)
r2 score:  0.5223842351893823


In [164]:
histparams={
    'max_iter' :[100,200, 300, 400], 
    'min_samples_leaf' : [5, 20, 40, 60], 
    'max_leaf_nodes' :[100, 200, 300, 400, 500, 600], 
    'l2_regularization' : [0.01, 0.1, 0.5, 0.9],
    'learning_rate':[0.01, 0.05, 0.1, 0.5, 0.9],
    
    }

In [165]:
histgb = HistGradientBoostingRegressor()
skboost = RandomizedSearchCV(estimator = histgb, 
                              param_distributions = histparams, 
                              n_iter = 100, 
                              n_jobs = 3, 
                              random_state =42,
                              cv =5,
                              verbose = 1)

skboost.fit(X_sel_tr, y_train2)

print("Best Hist Estimator: ", skboost.best_estimator_)
print("Hist r2 score: ", r2_score(y_validate2, skboost.best_estimator_.predict(X_sel_val))) 

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
F:\Anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  9.8min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 54.9min
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed: 124.5min
[Parallel(n_jobs=3)]: Done 500 out of 500 | elapsed: 149.5min finished


Best Hist Estimator:  HistGradientBoostingRegressor(l2_regularization=0.5, learning_rate=0.1,
                              loss='least_squares', max_bins=255,
                              max_depth=None, max_iter=100, max_leaf_nodes=500,
                              min_samples_leaf=40, n_iter_no_change=None,
                              random_state=None, scoring=None, tol=1e-07,
                              validation_fraction=0.1, verbose=0,
                              warm_start=False)
Hist r2 score:  0.41560470282183015


## Conclusion

##### Best model is:

In [168]:
best_regressor = randgrid.best_estimator_

In [166]:
randgrid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0.15,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=21, min_child_weight=15, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.01, reg_lambda=1, reg_lambda =0.05, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [172]:
X_sel_val.head(2)

,LocationLat^1,Severity^1 x LocationLat^1,Severity^1 x LocationLng^1,Severity^1 x Density(/sqmi)^1,Severity^1 x isWeekday^1,LocationLat^1 x LocationLng^1,LocationLat^1 x ZipCode^1,LocationLat^1 x Density(/sqmi)^1,LocationLat^1 x isWeekday^1,LocationLat^1 x SecondHalfHour^1,LocationLng^1 x ZipCode^1,LocationLng^1 x Density(/sqmi)^1,LocationLng^1 x Mon^1,LocationLng^1 x Noon(12-18)^1,ZipCode^1 x Mon^1,Density(/sqmi)^1 x Mon^1,Density(/sqmi)^1 x Noon(12-18)^1,Mon^1 x Autum^1,Mon^1 x Morning(6-12)^1,Spring^1 x Morning(6-12)^1
0,47.95,143.85,-366.91,1004.40,3.00,-5864.39,4708750.77,16053.38,47.95,0.00,-12010652.81,-40947.49,-122.30,-122.30,98203.00,334.80,334.80,1.00,0.00,0.00
1,47.69,143.06,-366.94,2612.70,3.00,-5832.73,4678800.68,41530.53,47.69,47.69,-12000744.80,-106522.43,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00


expected Mon^1 x Autum^1, Density(/sqmi)^1 x Mon^1, Severity^1 x isWeekday^1, LocationLat^1 x LocationLng^1, Density(/sqmi)^1 x Noon(12-18)^1, ZipCode^1 x Mon^1, Severity^1 x Density(/sqmi)^1, Spring^1 x Morning(6-12)^1, LocationLat^1 x Density(/sqmi)^1, LocationLat^1 x SecondHalfHour^1, LocationLng^1 x ZipCode^1, LocationLng^1 x Density(/sqmi)^1, Mon^1 x Morning(6-12)^1, Severity^1 x LocationLat^1, Severity^1 x LocationLng^1, LocationLat^1 x ZipCode^1, LocationLng^1 x Noon(12-18)^1, LocationLng^1 x Mon^1, LocationLat^1, LocationLat^1 x isWeekday^1 in input data

In [175]:
X_sel_val.columns

Index(['LocationLat^1', 'Severity^1 x LocationLat^1',
       'Severity^1 x LocationLng^1', 'Severity^1 x Density(/sqmi)^1',
       'Severity^1 x isWeekday^1', 'LocationLat^1 x LocationLng^1',
       'LocationLat^1 x ZipCode^1', 'LocationLat^1 x Density(/sqmi)^1',
       'LocationLat^1 x isWeekday^1', 'LocationLat^1 x SecondHalfHour^1',
       'LocationLng^1 x ZipCode^1', 'LocationLng^1 x Density(/sqmi)^1',
       'LocationLng^1 x Mon^1', 'LocationLng^1 x Noon(12-18)^1',
       'ZipCode^1 x Mon^1', 'Density(/sqmi)^1 x Mon^1',
       'Density(/sqmi)^1 x Noon(12-18)^1', 'Mon^1 x Autum^1',
       'Mon^1 x Morning(6-12)^1', 'Spring^1 x Morning(6-12)^1'],
      dtype='object')

In [177]:
columns = X_sel_val.columns

In [176]:
to_pred = [[47.95, 143.85, -366.91, 1004.40, 3.00, -5864.39, 4708750.77, 16053.38, 47.95, 0.00, -12010652.81, -40947.49, -122.30, -122.30, 98203.00, 334.80, 334.80, 1.00, 0.00, 0.00]]

In [179]:
to_pred_df = pd.DataFrame(to_pred, columns = columns )

In [180]:
best_regressor.predict(to_pred_df)

array([44.611904], dtype=float32)

In [212]:
with open("models/best_regressor.pickle", "wb") as pfile:
    pickle.dump(best_regressor, pfile)